# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme (EDITED BY JOHN and Guwei)

*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
%pip install sentence-transformers
%pip install faiss-cpu
%pip install faiss-gpu
%pip install datasets
%pip install gdown
%pip install accelerate
%pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [datasets]/13 [datasets]ess]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
import gdown
import os

# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# # File Path
# data_dir = "/content/drive/MyDrive/COMP90042-Data"


# URLs of the files to download
urls = [
    "https://drive.google.com/uc?id=1SIlHpjPhgr5NJpf6nK79aCevoOmvZgML",
    "https://drive.google.com/uc?id=1aTH-Zzq9dztxxXIPDW8MY3HZR_6pGbSt",
    "https://drive.google.com/uc?id=1iliVuUhHp2M48Svxl2FUa4hWaOxnnTo7",
    "https://drive.google.com/uc?id=1-AWc4xhd7YVV45tOti9Uu778x4YCsyeZ",
    "https://drive.google.com/uc?id=1zvWH6i6EQwTVgFlaKfNm6TXeo6DEEA9m"
]

# Corresponding filenames
filenames = [
    "evidence.json",
    "dev-claims.json",
    "train-claims.json",
    "test-claims-unlabelled.json",
    "dev-claims-baseline.json"
]

# Download each file
for url, filename in zip(urls, filenames):
    if not os.path.exists(filename):
        gdown.download(url, filename, quiet=False)
    else:
        print(f"{filename} already exists. Skipping download.")




Downloading...
From (original): https://drive.google.com/uc?id=1SIlHpjPhgr5NJpf6nK79aCevoOmvZgML
From (redirected): https://drive.google.com/uc?id=1SIlHpjPhgr5NJpf6nK79aCevoOmvZgML&confirm=t&uuid=5a94088f-55d0-41b0-a617-ec7b9b883df4
To: /workspace/evidence.json
100%|██████████| 174M/174M [00:15<00:00, 11.3MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1aTH-Zzq9dztxxXIPDW8MY3HZR_6pGbSt
To: /workspace/dev-claims.json
100%|██████████| 41.5k/41.5k [00:00<00:00, 396kB/s]
Downloading...
From: https://drive.google.com/uc?id=1iliVuUhHp2M48Svxl2FUa4hWaOxnnTo7
To: /workspace/train-claims.json
100%|██████████| 328k/328k [00:00<00:00, 896kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-AWc4xhd7YVV45tOti9Uu778x4YCsyeZ
To: /workspace/test-claims-unlabelled.json
100%|██████████| 24.3k/24.3k [00:00<00:00, 307kB/s]
Downloading...
From: https://drive.google.com/uc?id=1zvWH6i6EQwTVgFlaKfNm6TXeo6DEEA9m
To: /workspace/dev-claims-baseline.json
100%|██████████| 49.6k/49.6k [00:00<00:00

In [1]:
import faiss
from sentence_transformers import CrossEncoder
from sentence_transformers import InputExample, SentenceTransformer, losses
from torch.utils.data import DataLoader
import random
import torch
import pandas as pd
import json
import re

def retrieve(evi_ebds, claim_ebds, evi_df, claim_df, retrival_top_k, rerank_top_k,threshold_activated,score_threshold, cross_encoder,dev):
    embedding_dim = evi_ebds.shape[1]
    index = faiss.IndexFlatL2(embedding_dim)
    faiss.normalize_L2(evi_ebds)
    index.add(evi_ebds)
    retrieval = pd.DataFrame()
    retrieved_labels = []
    retrieved_evidences = []

    i = 0
    counts = 0
    claim_texts  =[]
    total = len(claim_ebds)
    for dev_claim_embedding in claim_ebds:
        faiss.normalize_L2(dev_claim_embedding.reshape(1, -1))
        D, I = index.search(dev_claim_embedding.reshape(1, -1), retrival_top_k)
        text = claim_df.iloc[i]['claim_text']
        pairs = [(text, evi_df['value'][a]) for a in I[0]]
        scores = cross_encoder.predict(pairs)
        reranked = sorted(zip(I[0], scores), key=lambda x: x[1], reverse=True)

        retrieved_evidence = []
        filtered = []
        first = True
        for idx, score in reranked[:rerank_top_k]:
            if first:
                evi_id = evi_df['ID'][idx]
                evi_content = evi_df['value'][idx]
                retrieved_evidence.append(evi_id)
                filtered.append((evi_id, score, evi_content))
                first = False
            else:
                if not threshold_activated or score >= score_threshold:
                    evi_id = evi_df['ID'][idx]
                    evi_content = evi_df['value'][idx]
                    retrieved_evidence.append(evi_id)
                    filtered.append((evi_id, score, evi_content))

        retrieved_evi =  [evi_df['ID'][a] for a in I[0]]
        retrieved_evidences.append(retrieved_evidence)
        retrieved_labels.append('SUPPORTS')
        claim_texts.append(text)

        if dev:
            print(f"Claim: {text}")
            if len(filtered) > 0:
                print("evidence relevant")
                for eid, score, c in filtered:
                    print(f"  {eid}, Score: {score:.4f} ")
            print(f"Ground truth: {claim_df.iloc[i]['evidences']}\n")

            count = 0
            for g in claim_df.iloc[i]['evidences']:
                if g in retrieved_evi:
                    count += 1
            counts += count / len(claim_df.iloc[i]['evidences'])



        print('Progress ', round(i * 100 / total, 3), '%')
        i += 1


    print("R: ", counts/ i)
    retrieval['ID'] = claim_df['ID']
    retrieval['evidences'] = retrieved_evidences
    retrieval['claim_label'] = retrieved_labels
    retrieval['claim_text'] = claim_texts
    return retrieval

def mine_hard_negatives(retrieved_evidences, ground_truth_evidences):
    negative_evidences = []
    for i in range(len(retrieved_evidences)):
        retrieved_evidence = retrieved_evidences.iloc[i]
        ground_truth_evidence = ground_truth_evidences.iloc[i]
        negative_evidence = []
        for e in retrieved_evidence:
            if e not in ground_truth_evidence:
                negative_evidence.append(int(re.findall(r'\d+', e)[0]))
        negative_evidences.append(negative_evidence)
    return negative_evidences

def load_train_data(train_dataframe, evidence_dateframe, random_negatives_amount_per_claim):
    train_data = []
    for id in range(len(train_dataframe)):
        claim_text =  train_dataframe.iloc[id]['claim_text']
        # print("CLAIM: ", claim_text)
        positive_evidence_ids = train_dataframe.iloc[id]['evidences_numeric_index']
        negative_evidence_ids = train_dataframe.iloc[id]['negative_evidences']
        for evid_id in positive_evidence_ids:

            evidence_text = evidence_dateframe.iloc[evid_id]['value']
            # print("POSITIVE: ", evidence_text)
            train_data.append(InputExample(texts=[claim_text, evidence_text], label=1.0))
        for ngevid_id in negative_evidence_ids:
            evidence_text = evidence_dateframe.iloc[ngevid_id]['value']
            # print("NEGATIVE: ", evidence_text)
            train_data.append(InputExample(texts=[claim_text, evidence_text], label=0.0))

        for i in range(random_negatives_amount_per_claim):
            neg_id = random.choice(list(set(evidence_dateframe.index) - set(positive_evidence_ids)))
            neg_text = evidence_dateframe.iloc[neg_id]['value']
            train_data.append(InputExample(texts=[claim_text, neg_text], label=0.0))
    return train_data

def load_positive_train_data(train_dataframe, evidence_dateframe):
    train_data = []
    for id in range(len(train_dataframe)):
        claim_text =  train_dataframe.iloc[id]['claim_text']
        positive_evidence_ids = train_dataframe.iloc[id]['evidences_numeric_index']
        for evid_id in positive_evidence_ids:
            evidence_text = evidence_dateframe.iloc[evid_id]['value']
            train_data.append(InputExample(texts=[claim_text, evidence_text], label=1.0))
    return train_data

def save_retrieval(retrieval, path):
    output = {}
    for i in range(len(retrieval)):
        output[retrieval.iloc[i]['ID']] = {}
        output[retrieval.iloc[i]['ID']]['evidences'] = retrieval.iloc[i]['evidences']
        output[retrieval.iloc[i]['ID']]['claim_label'] = retrieval.iloc[i]['claim_label']
        output[retrieval.iloc[i]['ID']]['claim_text'] = ''

    with open(path, 'w') as file:
        file.write(json.dumps(output))

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Read evidence dataset.

In [4]:
with open('evidence.json', 'r') as f:
    evidence = json.load(f)
flat_list = []
for key in evidence:
    flat_list.append({"ID": key, "value": evidence[key]})

evidence_df = pd.DataFrame(flat_list)

evidence_df.head()

,ID,value
0,evidence-0,"John Bennet Lawes, English entrepreneur and ag..."
1,evidence-1,Lindberg began his professional career at the ...
2,evidence-2,``Boston (Ladies of Cambridge)'' by Vampire We...
3,evidence-3,"Gerald Francis Goyer (born October 20, 1936) w..."
4,evidence-4,He detected abnormalities of oxytocinergic fun...


Read training and developing datasets

In [5]:
with open('train-claims.json', 'r') as f:
    train_df = json.load(f)
flat_list = []
for key in train_df:
    evidences_numeric_index = []
    for e in train_df[key]['evidences']:
        evidences_numeric_index.append(int(re.findall(r'\d+', e)[0]))
    flat_list.append({"ID": key, "claim_text": train_df[key]['claim_text'], "claim_label": train_df[key]['claim_label'], "evidences": train_df[key]['evidences'], "evidences_numeric_index": evidences_numeric_index})
train_df = pd.DataFrame(flat_list)

train_df.head()



,ID,claim_text,claim_label,evidences,evidences_numeric_index
0,claim-1937,Not only is there no scientific evidence that ...,DISPUTED,"[evidence-442946, evidence-1194317, evidence-1...","[442946, 1194317, 12171]"
1,claim-126,El Niño drove record highs in global temperatu...,REFUTES,"[evidence-338219, evidence-1127398]","[338219, 1127398]"
2,claim-2510,"In 1946, PDO switched to a cool phase.",SUPPORTS,"[evidence-530063, evidence-984887]","[530063, 984887]"
3,claim-2021,Weather Channel co-founder John Coleman provid...,DISPUTED,"[evidence-1177431, evidence-782448, evidence-5...","[1177431, 782448, 540069, 352655, 1007867]"
4,claim-2449,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO,"[evidence-1010750, evidence-91661, evidence-72...","[1010750, 91661, 722725, 554161, 430839]"


In [22]:
with open('dev-claims.json', 'r') as f:
    dev_df = json.load(f)
flat_list = []
for key in dev_df:
    evidences_numeric_index = []
    for e in dev_df[key]['evidences']:
        evidences_numeric_index.append(int(re.findall(r'\d+', e)[0]))
    flat_list.append({"ID": key, "claim_text": dev_df[key]['claim_text'], "claim_label": dev_df[key]['claim_label'], "evidences": dev_df[key]['evidences'], "evidences_numeric_index": evidences_numeric_index})
dev_df = pd.DataFrame(flat_list)

dev_df.head()

,ID,claim_text,claim_label,evidences,evidences_numeric_index
0,claim-752,[South Australia] has the most expensive elect...,SUPPORTS,"[evidence-67732, evidence-572512]","[67732, 572512]"
1,claim-375,when 3 per cent of total annual global emissio...,NOT_ENOUGH_INFO,"[evidence-996421, evidence-1080858, evidence-2...","[996421, 1080858, 208053, 699212, 832334]"
2,claim-1266,This means that the world is now 1C warmer tha...,SUPPORTS,"[evidence-889933, evidence-694262]","[889933, 694262]"
3,claim-871,"“As it happens, Zika may also be a good model ...",NOT_ENOUGH_INFO,"[evidence-422399, evidence-702226, evidence-28...","[422399, 702226, 286834, 472751, 641043]"
4,claim-2164,Greenland has only lost a tiny fraction of its...,REFUTES,"[evidence-52981, evidence-264761, evidence-947...","[52981, 264761, 947243, 424102]"


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

Calculate sentence embeddings for evidence dataset and trainning dataset.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

train_data = load_positive_train_data(train_df, evidence_df)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=10,
    output_path='./output-bi-encoder',
    weight_decay=0.01
)

evidence_embeddings = model.encode(evidence_df['value'], batch_size=64, show_progress_bar=True)
dev_claims_embeddings = model.encode(dev_df['claim_text'], batch_size=64, show_progress_bar=True)
train_claims_embeddings = model.encode(train_df['claim_text'], batch_size=64, show_progress_bar=True)


Step,Training Loss
500,0.904800
1000,0.670200
1500,0.528900
2000,0.439500
2500,0.386100


Batches:  20%|█▉        | 3687/18888 [01:29<04:37, 54.79it/s]

Fine tune cross encoder for re-ranking.

In [24]:
from datasets import Dataset
def load_positive_train_ds(train_dataframe, evidence_dateframe, t_ebds, e_ebds):
    query = []
    answer = []
    for id in range(len(train_dataframe)):
        claim_text =  train_dataframe.iloc[id]['claim_text']
        positive_evidence_ids = train_dataframe.iloc[id]['evidences_numeric_index']
        # claim_ebd = t_ebds[id]
        # none_added = True
        for evid_id in positive_evidence_ids:

            # evi_ebd = e_ebds[evid_id]
            evidence = evidence_dateframe.iloc[evid_id]['value']
            # similarities = model.similarity(claim_ebd, evi_ebd)

            # if similarities > 0:
            #     none_added = False
            # print('Claim: ', claim_text)
            # print('Evidence: ', evidence)

            query.append(claim_text)
            answer.append(evidence)
        # if none_added:
        #     query.append(claim_text)
        #     answer.append(evidence_dateframe.iloc[positive_evidence_ids[0]]['value'])

    return Dataset.from_dict({
    "query": query,
    "answer": answer,})


train_dataset = load_positive_train_ds(train_df, evidence_df, train_claims_embeddings, evidence_embeddings)
print(len(train_dataset))

4122


In [25]:
from sentence_transformers.cross_encoder.evaluation import CrossEncoderRerankingEvaluator
from sentence_transformers.cross_encoder import CrossEncoderTrainingArguments, CrossEncoderTrainer, losses
from sentence_transformers.util import mine_hard_negatives

eval_dataset = load_positive_train_ds(dev_df, evidence_df,dev_claims_embeddings, evidence_embeddings)
embedding_model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")
hard_eval_dataset = mine_hard_negatives(
    eval_dataset,
    embedding_model,
    corpus = train_dataset["answer"],  # Use the full dataset as the corpus
    num_negatives=50,  # How many negatives per question-answer pair
    batch_size=4096,  # Use a batch size of 4096 for the embedding model
    output_format="n-tuple",  # The output format is (query, positive, negative1, negative2, ...) for the evaluator
    include_positives=True,  # Key: Include the positive answer in the list of negatives
    use_faiss=True,  # Using FAISS is recommended to keep memory usage low (pip install faiss-gpu or pip install faiss-cpu)
)

reranking_evaluator = CrossEncoderRerankingEvaluator(
    samples=[
        {
            "query": sample["query"],
            "positive": [sample["answer"]],
            "documents": [sample[column_name] for column_name in hard_eval_dataset.column_names[2:]],
        }
        for sample in hard_eval_dataset
    ],
    batch_size=32,
    name="gooaq-dev",
)

Setting range_max to 55 based on the provided parameters.
Found 154 unique queries out of 491 total queries.
Found an average of 3.188 positives per query.


Querying FAISS index: 100%|██████████| 1/1 [00:00<00:00, 18.45it/s]


Metric       Positive       Negative     Difference
Count             491         24,550               
Mean           0.4280         0.4167         0.0114
Median         0.4312         0.4152         0.0053
Std            0.1499         0.1238         0.1363
Min            0.0165         0.1251        -0.5122
25%            0.3265         0.3309        -0.0782
50%            0.4312         0.4152         0.0053
75%            0.5316         0.5089         0.0986
Max            0.7839         0.8124         0.5326


In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

reranking_evaluator(cross_encoder)


model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
short_model_name = model_name if "/" not in model_name else model_name.split("/")[-1]
run_name = f"reranker-{short_model_name}-gooaq-cmnrl"
args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir=f"models/{run_name}",
    # Optional training parameters:
    num_train_epochs= 40,
    per_device_train_batch_size= 16,
    per_device_eval_batch_size= 16,
    learning_rate=1e-6,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="epoch",
    save_strategy="epoch",

    save_total_limit=2,
    run_name=run_name,  # Will be used in W&B if `wandb` is installed
    seed=12,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    weight_decay = 0.01,
)
loss = losses.MultipleNegativesRankingLoss(model=cross_encoder)

trainer = CrossEncoderTrainer(
    model=cross_encoder,
    args=args,
    train_dataset=train_dataset,
    loss=loss,
    evaluator=reranking_evaluator,
    eval_dataset=eval_dataset,
)
trainer.train()

# cross_encoder.fit(

#     train_dataloader=train_dataloader,
#     # loss_fct=train_loss,
#     epochs=20,
#     output_path='./fine-tuned-cross-encoder',
#     weight_decay=0.01
# )

Epoch,Training Loss,Validation Loss,Gooaq-dev Map,Gooaq-dev Mrr@10,Gooaq-dev Ndcg@10,Gooaq-dev Base Map,Gooaq-dev Base Mrr@10,Gooaq-dev Base Ndcg@10
1,No log,0.777211,0.287206,0.268836,0.350499,0.211680,0.200438,0.255458
2,0.945400,0.481745,0.285074,0.265625,0.346814,0.211680,0.200438,0.255458
3,0.945400,0.416315,0.286442,0.267307,0.350387,0.211680,0.200438,0.255458
4,0.541400,0.322416,0.295563,0.277129,0.361326,0.211680,0.200438,0.255458
5,0.541400,0.294034,0.301203,0.283518,0.368923,0.211680,0.200438,0.255458
6,0.405300,0.259070,0.305079,0.287356,0.373524,0.211680,0.200438,0.255458
7,0.405300,0.235486,0.306514,0.288865,0.376084,0.211680,0.200438,0.255458
8,0.342700,0.205107,0.308281,0.290812,0.379468,0.211680,0.200438,0.255458
9,0.342700,0.217836,0.310357,0.292545,0.380487,0.211680,0.200438,0.255458
10,0.286000,0.216622,0.311530,0.293901,0.382501,0.211680,0.200438,0.255458


TrainOutput(global_step=10320, training_loss=0.28073411985885266, metrics={'train_runtime': 2840.2285, 'train_samples_per_second': 58.052, 'train_steps_per_second': 3.634, 'total_flos': 0.0, 'train_loss': 0.28073411985885266, 'epoch': 40.0})

Second retrieve for testing.

In [11]:
# train_retrieval = retrieve(evidence_embeddings, train_claims_embeddings, evidence_df , train_df, 150, 4, True, 2, cross_encoder, True)
# save_retrieval(train_retrieval, 'train_retrieval.json')

In [30]:

dev_retrieval = retrieve(evidence_embeddings, dev_claims_embeddings, evidence_df , dev_df, 150, 5, True, 0, cross_encoder, True)

save_retrieval(dev_retrieval, 'dev_retrieval.json')


Claim: [South Australia] has the most expensive electricity in the world.
evidence relevant
  evidence-67732, Score: 7.8662 
  evidence-572512, Score: 7.7395 
  evidence-723533, Score: 5.3336 
  evidence-48256, Score: 5.1603 
  evidence-780332, Score: 4.8392 
Ground truth: ['evidence-67732', 'evidence-572512']

Progress  0.0 %
Claim: when 3 per cent of total annual global emissions of carbon dioxide are from humans and Australia prod­uces 1.3 per cent of this 3 per cent, then no amount of emissions reductio­n here will have any effect on global climate.
evidence relevant
  evidence-559290, Score: 4.4406 
  evidence-1182416, Score: 4.0809 
  evidence-430393, Score: 3.6325 
  evidence-415619, Score: 3.6274 
  evidence-215052, Score: 3.5522 
Ground truth: ['evidence-996421', 'evidence-1080858', 'evidence-208053', 'evidence-699212', 'evidence-832334']

Progress  0.649 %
Claim: This means that the world is now 1C warmer than it was in pre-industrial times
evidence relevant
  evidence-694262

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

I will do a classficiation, we were thinking of using transformer model then LSTM, and compare them and use the one with the best performance. 
For transformer, in this fact checking task, I will first try with the multi encoder fusion because we have a lot of cluses in the evidence and want to combine 


In [6]:
with open('test-claims-unlabelled.json','r') as f:
    data = f.read()
print(data[:200])

test_df = json.loads(data)
flat_list = []
for key in test_df:
    flat_list.append({"ID": key, "claim_text": test_df[key]['claim_text']})
tst_df = pd.DataFrame(flat_list)
tst_claims_embeddings = model.encode(tst_df['claim_text'], batch_size=64, show_progress_bar=True)
retrieval = retrieve(evidence_embeddings, tst_claims_embeddings, evidence_df , tst_df, 150, 5, False, 0, cross_encoder, False)
save_retrieval(retrieval, 'test-output.json')


{"claim-2967": {"claim_text": "The contribution of waste heat to the global climate is 0.028 W/m2."}, "claim-979": {"claim_text": "\u201cWarm weather worsened the most recent five-year drought, which 


Batches: 100%|██████████| 3/3 [00:00<00:00,  4.24it/s]


NameError: name 'evidence_embeddings' is not defined

In [33]:
import pprint

with open('test-output.json', 'r') as f:
    test_output = json.load(f)


    # Display a few entries from the test_output dictionary to see its structure

    # Show the first 3 items
    for i, (claim_id, result) in enumerate(test_output.items()):
        print(f"Claim ID: {claim_id}")
        pprint.pprint(result)
        print("-" * 60)
        if i >= 2:
            break

Claim ID: claim-2967
{'claim_label': 'SUPPORTS',
 'claim_text': '',
 'evidences': ['evidence-308923',
               'evidence-963856',
               'evidence-1185839',
               'evidence-43606',
               'evidence-219780']}
------------------------------------------------------------
Claim ID: claim-979
{'claim_label': 'SUPPORTS',
 'claim_text': '',
 'evidences': ['evidence-1141374',
               'evidence-844967',
               'evidence-957389',
               'evidence-111777',
               'evidence-346845']}
------------------------------------------------------------
Claim ID: claim-1609
{'claim_label': 'SUPPORTS',
 'claim_text': '',
 'evidences': ['evidence-52981',
               'evidence-766313',
               'evidence-264761',
               'evidence-44751',
               'evidence-691825']}
------------------------------------------------------------


## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*